In [59]:
import sklearn
from sklearn.datasets import load_files
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
#nltk.download('punkt')
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier

In [60]:
moviedir = 'movie_reviews'

# loading all files. 
movie = load_files(moviedir, shuffle=True)

In [77]:
# Split data into training and test sets
from sklearn.model_selection import train_test_split
docs_train, docs_test, y_train, y_test = train_test_split(movie.data, movie.target, 
                                                          test_size = 0.20, random_state = 12)

AttributeError: 'list' object has no attribute 'target_names'

In [105]:
text_clf = Pipeline([
    #('tfidf', TfidfVectorizer(min_df=2, tokenizer=nltk.word_tokenize, max_features=3000)),
    ('tfidf', TfidfVectorizer()),
    ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)),
    #'clf', MultinomialNB()),
])

text_clf.fit(docs_train, y_train)

import numpy as np
predicted = text_clf.predict(docs_test)
print("Accuracy ",np.mean(predicted == y_test))

Accuracy  0.8325


In [106]:
from sklearn import metrics
print(metrics.classification_report(y_test, predicted,
 target_names=['neg', 'pos']))

              precision    recall  f1-score   support

         neg       0.84      0.83      0.84       206
         pos       0.82      0.84      0.83       194

    accuracy                           0.83       400
   macro avg       0.83      0.83      0.83       400
weighted avg       0.83      0.83      0.83       400



In [80]:
print(metrics.confusion_matrix(y_test, predicted))

[[171  35]
 [ 32 162]]


In [ ]:
# parameter tuning

In [99]:
from sklearn.model_selection import GridSearchCV
parameters = {
 #'vect__ngram_range': [(1, 1)],
 'tfidf__use_idf': (True, False),
 'clf__alpha': (1e-2, 1e-3, 1e-4),
}

In [100]:
gs_clf = GridSearchCV(text_clf, parameters, cv=5, iid=False, n_jobs=-1)

In [101]:
gs_clf = gs_clf.fit(docs_train, y_train)

In [102]:
print(gs_clf.best_score_)

0.8349661434681979


In [103]:
for param_name in sorted(parameters.keys()):
 print("%s: %r" % (param_name, gs_clf.best_params_[param_name]))

clf__alpha: 0.001
tfidf__use_idf: True
